In [1]:
import geopandas as geo
import pandas as pd
from shapely.geometry import Point

In [28]:
df_first_try = pd.read_csv("Daten/rtr_datensatz_2019_2020_komplett.csv", sep=",")

## Load Final DataFrame

In [3]:
columns_needed = [
"network_type",
"lat",
"long",
"loc_accuracy",
"country_location",
"download_kbit",
"upload_kbit",
"ping_ms",
"lte_rsrp",
"lte_rsrq",
"platform",
"model",
"network_name",
"ndt_download_kbit",
"ndt_upload_kbit",
"implausible",
"signal_strength",
"pinned",
"dtm_level",
"signal_classification"]


In [4]:
df = pd.read_csv("Daten/rtr_datensatz_2019_2020_komplett.csv", sep=",", usecols = columns_needed, keep_default_na=False)

In [6]:
len(df)

4388792

In [5]:
df = df[(df["loc_accuracy"] < 150 ) & (df["country_location"] == "AT") #wie gross raster? 100 ok?
        & (df["implausible"] == False)]


In [6]:
len(df)  

3459380

In [7]:
df = df.drop(labels=['loc_accuracy', 'country_location', 'implausible'], axis=1)

In [8]:
gdf = geo.GeoDataFrame(
    df, geometry=geo.points_from_xy(df.long,df.lat),crs="EPSG:4326")
gdf = gdf.to_crs("EPSG:31287")
gdf = gdf.drop(labels=['lat', 'long'], axis=1)

In [9]:
gdf.reset_index(drop=True,inplace=True)

In [17]:

gdf.to_csv("points_rtr.csv")

In [10]:
from shapely.geometry import Polygon, Point
from shapely.strtree import STRtree
import numpy as np
import time
import gc

In [11]:
AUTmap = geo.read_file("Daten/raster_austria.gpkg")


In [12]:
Raster_austria_mit_Daten = geo.sjoin(AUTmap,gdf,how="left")

In [13]:
Raster_austria_mit_Daten.columns

Index(['id', 'left', 'top', 'right', 'bottom', 'geometry', 'index_right',
       'network_type', 'download_kbit', 'upload_kbit', 'ping_ms', 'lte_rsrp',
       'lte_rsrq', 'platform', 'model', 'network_name', 'ndt_download_kbit',
       'ndt_upload_kbit', 'signal_strength', 'pinned', 'dtm_level',
       'signal_classification'],
      dtype='object')

In [18]:
len(Raster_austria_mit_Daten)

3508391

In [31]:
(Raster_austria_mit_Daten.network_type.drop_duplicates())

0                 NaN
1                 LTE
1                WLAN
1               3G/4G
1        CELLULAR_ANY
1                 LAN
7               HSPA+
10             LTE CA
19             NR NSA
20              2G/4G
24               EDGE
39               UMTS
41              HSDPA
61              IWLAN
63       LTE NR avail
66                 NR
135             2G/3G
136          Ethernet
170                  
820          2G/3G/4G
3795             HSPA
17006           HSUPA
30350             GSM
33863       Bluetooth
Name: network_type, dtype: object

In [32]:
sum(Raster_austria_mit_Daten.network_type=="LTE")

558656

In [33]:
Raster_austria_mit_Daten['download_kbit'] = Raster_austria_mit_Daten['download_kbit'].fillna(0)


In [35]:
Raster_austria_mit_Daten_MAX = Raster_austria_mit_Daten[Raster_austria_mit_Daten.groupby(level=0)
                                                        .download_kbit.transform('max')
                                                        .eq(Raster_austria_mit_Daten.download_kbit)]


In [43]:
Raster_austria_mit_Daten_MIN = Raster_austria_mit_Daten[Raster_austria_mit_Daten.groupby(level=0)
                                                        .download_kbit.transform('min')
                                                        .eq(Raster_austria_mit_Daten.download_kbit)]


In [70]:
Raster_austria_mit_Daten_MEAN  = Raster_austria_mit_Daten

In [71]:
Raster_austria_mit_Daten_MEAN["Download_Mean"] = Raster_austria_mit_Daten_MEAN.groupby('id')["download_kbit"].transform('mean')


In [72]:
Raster_austria_mit_Daten_MEAN = Raster_austria_mit_Daten_MEAN.groupby("id").first()

In [77]:
len(Raster_austria_mit_Daten_MEAN)

85409

In [78]:
len(AUT)

85409

In [42]:
Raster_austria_mit_Daten_MAX.to_csv("Raster_austria_Download_MAX.csv")

In [45]:
Raster_austria_mit_Daten_MIN.to_csv("Raster_austria_Download_MIN.csv")

In [75]:
Raster_austria_mit_Daten_MEAN.to_csv("Raster_austria_Download_AVERAGE.csv")

In [19]:
AUTmap = geo.read_file("Daten/raster_austria.gpkg")
AUTmap['IndexListe'] = np.empty((len(AUTmap), 0)).tolist()